In [1]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - ------------------------------------- 0.5/12.8 MB 730.2 kB/s eta 0:00:17
     -- ------------------------------------ 0.8/12.8 MB 907.1 kB/s eta 0:00:14
     --- ------------------------------------ 1.0/12.8 MB 1.2 MB/s eta 0:00:11
     ---- ----------------------------------- 1.3/12.8 MB 1.2 MB/s eta 0:00:10
     ----- ---------------------------------- 1.8/12.8 MB 1.3 MB/s eta 0:00:09
     ------ --------------------------------- 2.1/12.8 MB 1.4 MB/s eta 0:00:08
     ------ --------------------------------- 2.1/12.8 MB 1.4 MB/s eta 0:00:08
     ------- -------------------------------- 2.4/12.8 MB 1.1 MB/s eta 0:00:10
     -------- ------------------------------- 2.6/12.8 MB 1.2 MB/s eta 0:00:09

### PDF_Extractor

In [2]:
import mysql.connector
import pdfplumber
import re
import numpy as np
import spacy

In [57]:
# Initialize the spacy model for organization and guarantor extraction
nlp = spacy.load("en_core_web_sm")

# Define patterns for CUSIP numbers and interest rates
# cusip_pattern = r'\b[0-9A-Z]{9}\b'
cusip_pattern = r'\b\d{5}[A-Z]{3}\d{1}\b'
interest_rate_pattern = r'\d+(\.\d+)?%'
# To get organizatin names using regex
# organization_pattern = r'\b\w+.*?(Inc|Ltd|Corporation|Corp|LLC|Bank|association|company|org|organization|pvt ltd)\b'
# organization_pattern = r'\b\w+\s\w*?\s*(?:' + keywords + r')\b'
organization_pattern = r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*\s(?:Inc|Ltd|LLC|Corporation|Corp|Co|Bank)\b'
# organization_pattern = r'\b([A-Z][a-z]+(?: [A-Z][a-z]+)* (?:|Inc\.|Ltd\.|Corporation|Corp\.|LLC\.|Bank))\b'
guarantor_pattern = r'guarantor(?: of)?(?: the)? ([A-Z][a-z]+(?: [A-Z][a-z]+)*)'

In [58]:
import glob  # To load files from directory

In [59]:
def extract_from_pdf(file_paths):
    data = []
    for file_path in file_paths:
        with pdfplumber.open(file_path) as pdf:
            for page_num, page in enumerate(pdf.pages):
                text = page.extract_text()
                metadata = pdf.metadata
                # Extract CUSIP numbers
                cusip_numbers = re.findall(cusip_pattern, text)
                # Extract interest rates
                interest_rates = re.findall(interest_rate_pattern, text) 
                # Find all matches in the text
                organizations = re.findall(organization_pattern, text)
                # Use NLP for organization names and guarantor
                doc = nlp(text)
                # organizations = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
                guarantors = re.findall(guarantor_pattern, text)
                # guarantors = [ent.text for ent in doc.ents if 'guarantor' in ent.sent.text.lower()]
                data.append({'cusip_numbers': ','.join(cusip_numbers) or None,
                            'interest_rates': ','.join(set(interest_rates)) or None,
                            'organizations': ','.join(organizations) or None,
                            'guarantors': ','.join(guarantors) or None,
                            'metadata': str(metadata) or None,
                            'page_num': page_num})
    return data


In [60]:
# MySQL Database connection
def connect_to_mysql():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Sulochana@522",
        database="pdf_extract_values"
    )
    return connection

In [61]:
# Insert extracted data into MySQL
def insert_data_to_mysql(data):
    connection = connect_to_mysql()
    cursor = connection.cursor()

    # Create table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS pdf_data (
            id INT AUTO_INCREMENT PRIMARY KEY,
            cusip_numbers TEXT,
            interest_rates TEXT,
            organizations TEXT,
            guarantors TEXT,
            metadata TEXT,
            page_num INT
        )
    """)

    # Prepare SQL query
    insert_query = """
        INSERT INTO pdf_data (cusip_numbers, interest_rates, organizations, guarantors, metadata, page_num)
        VALUES (%s, %s, %s, %s, %s, %s)
    """
    
    for record in data:
        cursor.execute(insert_query, (
            record['cusip_numbers'],
            record['interest_rates'],
            record['organizations'],
            record['guarantors'],
            record['metadata'],
            record['page_num']
        ))

    connection.commit()
    cursor.close()
    connection.close()

In [68]:
def extract_organizations(pdf_path):
    """Extracts organizations from a PDF file.

    Args:
        pdf_path: Path to the PDF file.

    Returns:
        List of extracted organizations.
    """
    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            text = page.extract_text()
            # Extract text from the PDF
            # text = extract_text(pdf_path)
            # Preprocess the text
            text = text.replace('\n', ' ').replace('\r', '')  # Remove newlines and carriage returns
            text = re.sub(r'[^\w\s]', '', text)  # Remove non-word characters
            text = text.lower()  # Convert to lowercase for case-insensitive matching
            # Define regular expressions for organization patterns
            organization_patterns = [r'(\w+\s?)+inc\.',r'(\w+\s?)+llc\.',
            r'(\w+\s?)+university',r'(\w+\s?)+college',
            r'(\w+\s?)+department',r'(\w+\s?)+agency',
            r'(\w+\s?)+office',r'(\w+\s?)+foundation',
            r'(\w+\s?)+association',r'(\w+\s?)+society',]

            # Extract potential organizations
            potential_organizations = []
            for pattern in organization_patterns:
                matches = re.findall(pattern, text)
                potential_organizations.extend(matches)
            # Filter and refine extracted organizations
            organizations = []
            for org in potential_organizations:
                if org not in organizations:  # Remove duplicates
                    organizations.append(org)

    return organizations

In [ ]:
extract_organizations("Sample_1.pdf")

In [63]:
if __name__ == "__main__":
    # Get PDF files from directory (You can also pass file names as arguments)
    pdf_files = glob.glob("*.pdf")

    if len(pdf_files) == 0:
        print("No PDF files found in the directory!")
    else:
        extracted_data = extract_from_pdf(pdf_files)
        insert_data_to_mysql(extracted_data)
        # print(extracted_data)

In [17]:
## Single file execution at a time

# # Usage
# file_path = "Sample_2.pdf"
# extracted_data = extract_from_pdf(file_path)
# insert_data_to_mysql(extracted_data)
# extracted_data[2]